In [1]:
import datetime
from ib_insync import *
import pandas as pd

import sys
from pathlib import Path

# Get the parent directory of the current notebook's directory
project_root = Path.cwd().parent

# Add the project root to the system path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

try:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db
except:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db

In [2]:
ib = connect_to_IB(clientid=2)

# Develop a method to retrieve the current portfolio as dataframe and store it in arcticdb

In [3]:
pm = PortfolioManager(ib)

In [4]:
pm.fetch_portfolio()

/Users/johanneslange/Desktop/Pythonprojekte/IB-Multi-Strategy-ATS/broker/portfolio.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([portfolio_df, cash_df], ignore_index=True)


,symbol,contractType,position,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL,conId,lastTradeDate,account
0,EWT,STK,200.0000,44.210701,8842.1400,44.256828,-9.23,0.0,253190576,,U7706434
1,IAU,STK,300.0000,38.399899,11519.9700,38.454255,-16.31,0.0,474829650,,U7706434
2,IAU,OPT,-2.0000,0.310035,-62.0100,23.350200,-15.31,0.0,671974250,20240216,U7706434
3,INDA,STK,100.0000,49.571800,4957.1800,50.000000,-42.82,0.0,101484826,,U7706434
4,IUSQ,STK,100.0000,69.320000,6932.0000,65.602785,371.72,0.0,239363241,,U7706434
5,IWM,STK,100.0000,192.940002,19294.0000,199.616920,-667.69,0.0,9579970,,U7706434
6,IWM,OPT,-1.0000,2.167463,-216.7500,183.368900,-33.38,0.0,637114226,20240216,U7706434
7,SOXX,OPT,-1.0000,1.382843,-138.2800,699.944800,561.66,0.0,653437229,20240216,U7706434
8,TLT,STK,100.0000,94.230003,9423.0000,99.051625,-482.16,0.0,15547841,,U7706434
9,TLT,OPT,-1.0000,0.882805,-88.2800,76.199700,-12.08,0.0,614854428,20240216,U7706434


In [5]:
[item for item in ib.portfolio()[0]]

[Stock(conId=253190576, symbol='EWT', right='0', primaryExchange='ARCA', currency='USD', localSymbol='EWT', tradingClass='EWT'),
 200.0,
 44.210701,
 8842.14,
 44.2568285,
 -9.23,
 0.0,
 'U7706434']

In [6]:
ib.portfolio()[0]

PortfolioItem(contract=Stock(conId=253190576, symbol='EWT', right='0', primaryExchange='ARCA', currency='USD', localSymbol='EWT', tradingClass='EWT'), position=200.0, marketPrice=44.210701, marketValue=8842.14, averageCost=44.2568285, unrealizedPNL=-9.23, realizedPNL=0.0, account='U7706434')

In [26]:
# pd.set_option('display.float_format', '{:.2f}'.format)

total_equity =  sum(float(entry.value) for entry in ib.accountSummary() if entry.tag == "EquityWithLoanValue")
portfolio_data = []

for item in ib.portfolio():
    symbol = item.contract.symbol
    contractType = item.contract.secType

    if contractType == "OPT":
        right = "Call" if item.contract.right == "C" else "Put"
        asset_class = right + " " + str(item.contract.strike) + " " + item.contract.lastTradeDateOrContractMonth
        if item.position < 0:
            pnl = ((item.marketPrice/(item.averageCost/100)) -1) * (-1)
        else:
            pnl = ( (item.marketPrice/ (item.averageCost/100)) -1)
    elif contractType == "FUT":
        asset_class = item.contract.localSymbol + " " + item.contract.lastTradeDateOrContractMonth
        pnl = ((item.marketPrice/(item.averageCost)) -1)
        pnl = pnl *(-1) if item.position < 0 else pnl
    elif contractType == "STK":
        pnl = ((item.marketPrice/(item.averageCost)) -1)
        pnl = pnl *(-1) if item.position < 0 else pnl
        asset_class = contractType
    
    position_dict = {'timestamp': datetime.datetime.now(),
                    'symbol': symbol,
                     'asset class': asset_class,
                     'position':item.position,
                     '% of nav':item.marketValue/total_equity,
                     'averageCost': item.averageCost,
                     'marketPrice': item.marketPrice,
                     'pnl %': pnl,
                     'strategy': '',
                     'contract': str(item.contract),
                     'marketValue': item.marketValue,
                     'unrealizedPNL': item.unrealizedPNL,
                     'realizedPNL': item.realizedPNL,
                     'account': item.account}
    
    portfolio_data.append(position_dict)
        
df = pd.DataFrame(portfolio_data)
df

,timestamp,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,marketValue,unrealizedPNL,realizedPNL,account
0,2024-01-21 15:27:35.463259,EWT,STK,200.0,0.087861,44.256828,44.210701,-0.001042,,"Stock(conId=253190576, symbol='EWT', right='0'...",8842.14,-9.23,0.0,U7706434
1,2024-01-21 15:27:35.463340,IAU,STK,300.0,0.114469,38.454255,38.399899,-0.001414,,"Stock(conId=474829650, symbol='IAU', right='0'...",11519.97,-16.31,0.0,U7706434
2,2024-01-21 15:27:35.463369,IAU,Call 39.0 20240216,-2.0,-0.000616,23.350200,0.310035,-0.327763,,"Option(conId=671974250, symbol='IAU', lastTrad...",-62.01,-15.31,0.0,U7706434
3,2024-01-21 15:27:35.463389,INDA,STK,100.0,0.049257,50.000000,49.571800,-0.008564,,"Stock(conId=101484826, symbol='INDA', right='0...",4957.18,-42.82,0.0,U7706434
4,2024-01-21 15:27:35.463401,IUSQ,STK,100.0,0.068880,65.602785,69.320000,0.056662,,"Stock(conId=239363241, symbol='IUSQ', right='0...",6932.00,371.72,0.0,U7706434
5,2024-01-21 15:27:35.463412,IWM,STK,100.0,0.191717,199.616920,192.940002,-0.033449,,"Stock(conId=9579970, symbol='IWM', right='0', ...",19294.00,-667.69,0.0,U7706434
6,2024-01-21 15:27:35.463424,IWM,Call 199.0 20240216,-1.0,-0.002154,183.368900,2.167463,-0.182023,,"Option(conId=637114226, symbol='IWM', lastTrad...",-216.75,-33.38,0.0,U7706434
7,2024-01-21 15:27:35.463441,SOXX,Put 525.0 20240216,-1.0,-0.001374,699.944800,1.382843,0.802435,,"Option(conId=653437229, symbol='SOXX', lastTra...",-138.28,561.66,0.0,U7706434
8,2024-01-21 15:27:35.463453,TLT,STK,100.0,0.093632,99.051625,94.230003,-0.048678,,"Stock(conId=15547841, symbol='TLT', right='0',...",9423.00,-482.16,0.0,U7706434
9,2024-01-21 15:27:35.463465,TLT,Call 96.0 20240216,-1.0,-0.000877,76.199700,0.882805,-0.158541,,"Option(conId=614854428, symbol='TLT', lastTrad...",-88.28,-12.08,0.0,U7706434


In [27]:

latest_positions = df.sort_values(by='timestamp').groupby(['symbol', 'strategy', 'asset class']).last().reset_index()
latest_positions

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,marketValue,unrealizedPNL,realizedPNL,account
0,EWT,,STK,2024-01-21 15:27:35.463259,200.0,0.087861,44.256828,44.210701,-0.001042,"Stock(conId=253190576, symbol='EWT', right='0'...",8842.14,-9.23,0.0,U7706434
1,IAU,,Call 39.0 20240216,2024-01-21 15:27:35.463369,-2.0,-0.000616,23.350200,0.310035,-0.327763,"Option(conId=671974250, symbol='IAU', lastTrad...",-62.01,-15.31,0.0,U7706434
2,IAU,,STK,2024-01-21 15:27:35.463340,300.0,0.114469,38.454255,38.399899,-0.001414,"Stock(conId=474829650, symbol='IAU', right='0'...",11519.97,-16.31,0.0,U7706434
3,INDA,,STK,2024-01-21 15:27:35.463389,100.0,0.049257,50.000000,49.571800,-0.008564,"Stock(conId=101484826, symbol='INDA', right='0...",4957.18,-42.82,0.0,U7706434
4,IUSQ,,STK,2024-01-21 15:27:35.463401,100.0,0.068880,65.602785,69.320000,0.056662,"Stock(conId=239363241, symbol='IUSQ', right='0...",6932.00,371.72,0.0,U7706434
5,IWM,,Call 199.0 20240216,2024-01-21 15:27:35.463424,-1.0,-0.002154,183.368900,2.167463,-0.182023,"Option(conId=637114226, symbol='IWM', lastTrad...",-216.75,-33.38,0.0,U7706434
6,IWM,,STK,2024-01-21 15:27:35.463412,100.0,0.191717,199.616920,192.940002,-0.033449,"Stock(conId=9579970, symbol='IWM', right='0', ...",19294.00,-667.69,0.0,U7706434
7,SOXX,,Put 525.0 20240216,2024-01-21 15:27:35.463441,-1.0,-0.001374,699.944800,1.382843,0.802435,"Option(conId=653437229, symbol='SOXX', lastTra...",-138.28,561.66,0.0,U7706434
8,TLT,,Call 96.0 20240216,2024-01-21 15:27:35.463465,-1.0,-0.000877,76.199700,0.882805,-0.158541,"Option(conId=614854428, symbol='TLT', lastTrad...",-88.28,-12.08,0.0,U7706434
9,TLT,,Call 98.0 20240216,2024-01-21 15:27:35.463490,-1.0,-0.000400,73.369800,0.402930,0.450823,"Option(conId=614854493, symbol='TLT', lastTrad...",-40.29,33.08,0.0,U7706434


# Add some data to portfolio

In [28]:
ac = initialize_db('db')
ac.list_libraries()
# general = ac.get_library('general')
portfolio_lib = ac.get_library('portfolio')
portfolio_lib.list_symbols()

[]

In [29]:
dummy_data = df.loc[0,:].copy()
dummy_data['position'] = dummy_data['position'] / 2
dummy_data['% of nav'] = dummy_data['% of nav'] / 2
dummy_data['averageCost'] = dummy_data['averageCost'] * 0.9
dummy_data['unrealizedPNL'] = (dummy_data['marketPrice'] - dummy_data['averageCost']) * dummy_data['position']
dummy_data['marketValue'] = dummy_data['marketPrice'] * dummy_data['position']
if dummy_data['asset class'] == 'STK' or dummy_data['asset class'] == 'FUT':
    pnl = ((dummy_data.marketPrice/(dummy_data.averageCost)) -1)
    pnl = pnl *(-1) if dummy_data.position < 0 else pnl
else:
    if dummy_data.position < 0:
        pnl = ((dummy_data.marketPrice/(dummy_data.averageCost/100)) -1) * (-1)
    else:
        pnl = ( (dummy_data.marketPrice/ (dummy_data.averageCost/100)) -1)
dummy_data['pnl %'] = pnl
dummy_data = pd.DataFrame(dummy_data).T


In [31]:
# Convert 'contract' column to string as ArcticDB may not handle custom objects well
dummy_data['contract'] = dummy_data['contract'].astype(str)

# Ensure that all other columns are of a consistent and compatible data type
# For example, convert all numeric columns to float
numeric_columns = ['position', '% of nav', 'averageCost', 'marketPrice', 'pnl %', 'marketValue', 'unrealizedPNL', 'realizedPNL']
for col in numeric_columns:
    dummy_data[col] = pd.to_numeric(dummy_data[col], errors='coerce')

# Now write the modified DataFrame to ArcticDB
portfolio_lib.write('positions', dummy_data)


VersionedItem(symbol='positions', library='portfolio', data=n/a, version=0, metadata=None, host='S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb)')

In [32]:
df_ac = portfolio_lib.read('positions').data
df_ac

,timestamp,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,marketValue,unrealizedPNL,realizedPNL,account
0,2024-01-21 15:27:35.463259,EWT,STK,100.0,0.04393,39.831146,44.210701,0.109953,,"Stock(conId=253190576, symbol='EWT', right='0'...",4421.0701,437.955535,0.0,U7706434


#### Now, we will get ib positions and match them with arcticdb entries

In [33]:
# pd.set_option('display.float_format', '{:.2f}'.format)

total_equity =  sum(float(entry.value) for entry in ib.accountSummary() if entry.tag == "EquityWithLoanValue")
portfolio_data = []

for item in ib.portfolio():
    symbol = item.contract.symbol
    contractType = item.contract.secType

    if contractType == "OPT":
        right = "Call" if item.contract.right == "C" else "Put"
        asset_class = right + " " + str(item.contract.strike) + " " + item.contract.lastTradeDateOrContractMonth
        if item.position < 0:
            pnl = ((item.marketPrice/(item.averageCost/100)) -1) * (-1)
        else:
            pnl = ( (item.marketPrice/ (item.averageCost/100)) -1)
    elif contractType == "FUT":
        asset_class = item.contract.localSymbol + " " + item.contract.lastTradeDateOrContractMonth
        pnl = ((item.marketPrice/(item.averageCost)) -1)
        pnl = pnl *(-1) if item.position < 0 else pnl
    elif contractType == "STK":
        pnl = ((item.marketPrice/(item.averageCost)) -1)
        pnl = pnl *(-1) if item.position < 0 else pnl
        asset_class = contractType
    
    position_dict = {'timestamp': datetime.datetime.now(),
                    'symbol': symbol,
                     'asset class': asset_class,
                     'position':item.position,
                     '% of nav':item.marketValue/total_equity,
                     'averageCost': item.averageCost,
                     'marketPrice': item.marketPrice,
                     'pnl %': pnl,
                     'strategy': '',
                     'contract': str(item.contract),
                     'marketValue': item.marketValue,
                     'unrealizedPNL': item.unrealizedPNL,
                     'realizedPNL': item.realizedPNL,
                     'account': item.account}
    
    portfolio_data.append(position_dict)
        
df_ib = pd.DataFrame(portfolio_data)
df_ib

,timestamp,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,marketValue,unrealizedPNL,realizedPNL,account
0,2024-01-21 15:35:48.881661,EWT,STK,200.0,0.087861,44.256828,44.210701,-0.001042,,"Stock(conId=253190576, symbol='EWT', right='0'...",8842.14,-9.23,0.0,U7706434
1,2024-01-21 15:35:48.881784,IAU,STK,300.0,0.114469,38.454255,38.399899,-0.001414,,"Stock(conId=474829650, symbol='IAU', right='0'...",11519.97,-16.31,0.0,U7706434
2,2024-01-21 15:35:48.881816,IAU,Call 39.0 20240216,-2.0,-0.000616,23.350200,0.310035,-0.327763,,"Option(conId=671974250, symbol='IAU', lastTrad...",-62.01,-15.31,0.0,U7706434
3,2024-01-21 15:35:48.881844,INDA,STK,100.0,0.049257,50.000000,49.571800,-0.008564,,"Stock(conId=101484826, symbol='INDA', right='0...",4957.18,-42.82,0.0,U7706434
4,2024-01-21 15:35:48.881861,IUSQ,STK,100.0,0.068880,65.602785,69.320000,0.056662,,"Stock(conId=239363241, symbol='IUSQ', right='0...",6932.00,371.72,0.0,U7706434
5,2024-01-21 15:35:48.881876,IWM,STK,100.0,0.191717,199.616920,192.940002,-0.033449,,"Stock(conId=9579970, symbol='IWM', right='0', ...",19294.00,-667.69,0.0,U7706434
6,2024-01-21 15:35:48.881892,IWM,Call 199.0 20240216,-1.0,-0.002154,183.368900,2.167463,-0.182023,,"Option(conId=637114226, symbol='IWM', lastTrad...",-216.75,-33.38,0.0,U7706434
7,2024-01-21 15:35:48.881916,SOXX,Put 525.0 20240216,-1.0,-0.001374,699.944800,1.382843,0.802435,,"Option(conId=653437229, symbol='SOXX', lastTra...",-138.28,561.66,0.0,U7706434
8,2024-01-21 15:35:48.881933,TLT,STK,100.0,0.093632,99.051625,94.230003,-0.048678,,"Stock(conId=15547841, symbol='TLT', right='0',...",9423.00,-482.16,0.0,U7706434
9,2024-01-21 15:35:48.881950,TLT,Call 96.0 20240216,-1.0,-0.000877,76.199700,0.882805,-0.158541,,"Option(conId=614854428, symbol='TLT', lastTrad...",-88.28,-12.08,0.0,U7706434


In [ ]:
def match_ib_positions_with_arcticdb():
    library = ac.get_library('portfolio',create_if_missing=True)
    df_ac = library.read('positions').data
    latest_positions = df_ac.sort_values(by='timestamp').groupby(['symbol', 'strategy', 'asset class']).last().reset_index()
    df_ib =  df_ib # self.get_positions_from_ib()

In [49]:
df_ac = portfolio_lib.read('positions').data
latest_positions_in_ac = df_ac.sort_values(by='timestamp').groupby(['symbol', 'strategy', 'asset class']).last().reset_index()
latest_positions_in_ac

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,marketValue,unrealizedPNL,realizedPNL,account
0,EWT,,STK,2024-01-21 15:27:35.463259,100.0,0.04393,39.831146,44.210701,0.109953,"Stock(conId=253190576, symbol='EWT', right='0'...",4421.0701,437.955535,0.0,U7706434


In [ ]:
# Assuming df_ib and latest_positions_in_ac are your DataFrames from IB and ArcticDB

for _, row in df_ib.iterrows():
    symbol = row['symbol']
    asset_class = row['asset class']
    total_position = row['position']
    total_average_cost = row['averageCost']
    total_market_price = row['marketPrice']

    strategy_rows = latest_positions_in_ac[(latest_positions_in_ac['symbol'] == symbol) & (latest_positions_in_ac['asset class'] == asset_class)]

    if not strategy_rows.empty:
        assigned_position = strategy_rows['position'].sum()
        residual_position = total_position - assigned_position

        if len(strategy_rows) == 1 and total_position == strategy_rows.iloc[0]['position']:
            # Case 1: Single strategy row and matching position
            # Update all columns in ArcticDB for this symbol & asset class combination except 'strategy'
            # Example: Update marketPrice and other related fields in ArcticDB
            pass

        else:
            # Case 2: Single/Multiple strategy rows and differing position
            # Calculate averageCost for the residual position
            assigned_average_cost = strategy_rows['averageCost'].iloc[0] if len(strategy_rows) == 1 else calculate_weighted_average_cost(strategy_rows)
            residual_average_cost = calculate_residual_average_cost(total_average_cost, total_position, assigned_average_cost, assigned_position)

            # Update marketPrice, marketValue, % of NAV, pnl%, unrealizedPNL for the assigned positions in ArcticDB
            # And calculate the corresponding values for the residual position
            pass

def calculate_weighted_average_cost(strategy_rows):
    # Implement logic to calculate a weighted average cost for multiple strategy rows
    pass

def calculate_residual_average_cost(total_average_cost, total_position, assigned_average_cost, assigned_position):
    # Implement logic to calculate the average cost for the residual position
    return ((total_average_cost * total_position) - (assigned_average_cost * assigned_position)) / (total_position - assigned_position)

# Further implement the functions or logic needed in 'pass' sections

In [57]:
for _, row in df_ib.iterrows():
    symbol = row.symbol
    asset_class = row['asset class']
    total_position = row['position']

    strategy_rows = latest_positions_in_ac[(latest_positions_in_ac['symbol'] == symbol) & (latest_positions_in_ac['asset class'] == asset_class)]

    if not strategy_rows.empty:
        if len(strategy_rows) == 1 and total_position == strategy_rows.position:
            # update all cols in ac for this symbol & asset class combination except col strategies
            pass
        if len(strategy_rows) == 1 and total_position != strategy_rows.position:
            # 1. update cols marketPrice in ac from ib query and calculate new values for 'marketValue' (position*marketPrice), '% of nav', 'pnl%','unrealized'
            # calculate averageCost for the residual position (ib total averageCost = ac.averageCost * (ac.position/total_position) + residual.averageCost * (residual.position/total_position))
            # with the averageCost value and the position for the residual we can calculate 'marketValue', '% of nav', 'pnl%','unrealized'

            # if possible generalize the above for the cases where len(strategy_rows) > 1
            pass


1
EWT in ac
100.0 100.0


In [42]:
latest_positions['symbol'].values

array(['EWT'], dtype=object)